# Map of Gordon's Bay

## For Prodive's Divemaster course

As part of this course, I need to draw a map of a dive site. Being an odd fish, I've got a bit carried away. There's a lot more context in the [readme file](https://github.com/notionparallax/dive-map/blob/main/README.md) in the repo.

Let's get started. If you're just following along for the pictures, ignore all the python (the coloured writing) and go straight to the pictures.


In [ ]:
import os
from functools import partial

import contextily as cx
import folium
import geopandas as gp
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import matplotlib.colors
import numpy as np
import pandas as pd
import pytz
from geopy import Point as geopy_pt
from geopy.distance import geodesic
from mpl_toolkits.axes_grid1 import make_axes_locatable
from shapely import centroid
from shapely.geometry import LineString, MultiPoint, Point, Polygon
from shapely.ops import voronoi_diagram


from map_functions import (
    add_note_label,
    add_numbered_marker_label,
    add_tolerance_circle,
    depth_to_colour,
    draw_north_arrow,
    draw_scale_bar,
    draw_shortcut_arrow,
    get_gps_data_single_dive,
    get_photo_data,
    make_depth_df,
    make_dive_df,
    make_marker_text,
    naieve_ffill,
    plot_gps_trace,
    measure_line_string,
    move_pt,
    prep_for_contour,
)

In [ ]:
# I've got the photo data as python data, rather than json, not really for any good reason though. It's just a big list of dictionaries.
from photo_meta import photo_meta
from photo_meta_day_2 import photo_meta as photo_meta_day_2

photo_meta = photo_meta + photo_meta_day_2

In [ ]:
plt.rcParams["svg.fonttype"] = "none"
TEXT_COLOUR = "white"
CRS = "EPSG:4326"
X_OFFSET = 0.0001
X_OFFSET_SMALL = 0.0002
Y_OFFSET = 0.000015

The depth data comes from my watch, a suunto D5, and I export the `.fit` file from the phone app.


In [ ]:
depth_df_1 = make_depth_df(
    [
        os.path.join("fit", "ScubaDiving_2024-03-08T09_29_45.fit"),
        os.path.join("fit", "ScubaDiving_2024-03-08T11_26_21.fit"),
    ]
)
depth_df_1.plot(
    title="Depth of the dives\n 1 around the gordon's chain, 2 around the boulder garden",
    ylabel="Depth (m)",
    xlabel="Time (UTC)",
)

In [ ]:
depth_df_2 = make_depth_df(
    [
        os.path.join("fit", "ScubaDiving_2024-03-23T08_51_29.fit"),
        os.path.join("fit", "ScubaDiving_2024-03-23T09_41_59.fit"),
        os.path.join("fit", "ScubaDiving_2024-03-23T11_06_51.fit"),
    ]
)
depth_df_2.plot(
    title=(
        "Depth of the dives\n "
        "1 around bottom of the wall/desert interface,\n"
        "2 across to the other side"
    ),
    ylabel="Depth (m)",
    xlabel="Time (UTC)",
)

depth_df = pd.concat([depth_df_1, depth_df_2])
# depth_df.plot() #This is boring because there's 2 weeks of empty space between dive one and dive 2

The GPS data comes from an app on my phone. This odd shape is because I forgot to turn it off and then we drove back to the shop. If we look at the numbers on the x axis, we've almost gone 0.1 of a degree, which in metric is:

```
Decimal Places   Aprox. Distance    Say What?
1                10 kilometers      6.2 miles
2                1 kilometer        0.62 miles
3                100 meters         About 328 feet
4                10 meters          About 33 feet
5                1 meter            About 3 feet
6                10 centimeters     About 4 inches
7                1.0 centimeter     About 1/2 an inch
8                1.0 millimeter     The width of paperclip wire.
9                0.1 millimeter     The width of a strand of hair.
10               10 microns         A speck of pollen.
11               1.0 micron         A piece of cigarette smoke.
12               0.1 micron         You're doing virus-level mapping at this point.
13               10 nanometers      Does it matter how big this is?
14               1.0 nanometer      Your fingernail grows about this far in one second.
15               0.1 nanometer      An atom. An atom! What are you mapping?
```

from [here](https://gis.stackexchange.com/questions/8650/measuring-accuracy-of-latitude-and-longitude)

But we don't care about driving around the Eastern Subburbs, so we'll have to crop it off. Also, this is both dives, so we'll need to split those out too.


In [ ]:
first_dive_day = os.path.join("gps", "20240308-090746 - Gordons.gpx")
plot_gps_trace(fp=first_dive_day)

In [ ]:
day_2_dive_1 = os.path.join("gps", "20240323-081550 - Map dive Saturday morning.gpx")
plot_gps_trace(fp=day_2_dive_1)

In [ ]:
day_2_dive_2 = os.path.join("gps", "20240323-104518 - Dive 2.gpx")
plot_gps_trace(fp=day_2_dive_2)

In [ ]:
fp = day_2_dive_1
recorded_path = gp.read_file(fp, layer="tracks")
ax = recorded_path.plot()

The following cells show the dives, but with only the dive data shown:


In [ ]:
dives_lon_lat_time_day_1a = get_gps_data_single_dive(
    first_dive_day,
    description="chain_loop",
    crop=True,
    dive_end_time_delta=180,
    dive_start_time_delta=250,
    end_time="2024-03-08T02:25:26Z",
)
dives_gdf_1a = make_dive_df(dives_lon_lat_time_day_1a)
dives_gdf_1a.plot()

In [ ]:
dives_lon_lat_time_day_1b = get_gps_data_single_dive(
    first_dive_day,
    description="boulder_garden",
    crop=True,
    end_time="2024-03-08T02:25:26Z",
    dive_end_time_delta=70,
    dive_start_time_delta=120,
)
dives_gdf_1b = make_dive_df(dives_lon_lat_time_day_1b)
dives_gdf_1b.plot()

In [ ]:
dives_lon_lat_time_day_2a = get_gps_data_single_dive(
    day_2_dive_1,
    description="wall_to_desert",
    crop=True,
    end_time="2024-03-22 22:52:06+00:00",
    dive_end_time_delta=5,
    dive_start_time_delta=65,
)
# ((dives_gdf_2b.index.max()-dives_gdf_2b.index.min()).total_seconds() / 60)-15,
dives_gdf_2a = make_dive_df(dives_lon_lat_time_day_2a)
dives_gdf_2a.plot()

In [ ]:
dives_lon_lat_time_day_2b = get_gps_data_single_dive(
    day_2_dive_2,
    description="far_side_desert",
    crop=True,
    end_time="2024-03-23 01:14:03.999000+00:00",
    dive_end_time_delta=10,
    dive_start_time_delta=68,
)
dives_gdf_2b = make_dive_df(dives_lon_lat_time_day_2b)
dives_gdf_2b.plot()

In [ ]:
dives_gdf = gp.GeoDataFrame(
    pd.concat(
        [
            dives_gdf_1a,
            dives_gdf_1b,
            dives_gdf_2a,
            dives_gdf_2b,
        ]
    )
)
dives_gdf.plot(column="description")
plt.title(f"All dives so far ({', '.join( dives_gdf.description.unique())})")

The glue that joins it all up is photos, the photos tell us when we were at a feature. And in this case, what the bottom condition was like.


In [ ]:
photo_df = get_photo_data()

There are some tricky bits because everything is in different time zones, so let's unify them all into utc:


In [ ]:
print("Before:")
print("\tdives_df:", repr(dives_gdf.iloc[0].name))
print("\tdepth_df:", repr(depth_df.iloc[0].name))
print("\tphoto_df:", repr(photo_df.iloc[0].name))

# Convert all timestamps to UTC
dives_gdf.index = dives_gdf.index.tz_convert("UTC")
depth_df_1.index = depth_df_1.index.tz_convert("UTC")
photo_df.index = photo_df.index.tz_convert("UTC")

print("After:")
print("\tdives_df:", repr(dives_gdf.iloc[0].name))
print("\tdepth_df:", repr(depth_df.iloc[0].name))
print("\tphoto_df:", repr(photo_df.iloc[0].name))

dives_gdf["source"] = "dives"
depth_df["source"] = "depth"
photo_df["source"] = "photo"

If for some reason the data is too heavy, use this to chop it down. It's not doing any chopping at the moment.


In [ ]:
reduced_dives = dives_gdf
# reduced_dives = reduced_dives.iloc[::60]  # pick one frame a minute
# reduced_dives = reduced_dives[
#     reduced_dives.index > depth_df.index[0]
# ]  # wait until there's depth data
all_df = pd.concat([reduced_dives, depth_df, photo_df])
all_df.sort_index(axis=0, inplace=True)
# temp_df = all_df.copy(deep=True)
# temp_df.index = temp_df.index.tz_localize(None)
# temp_df.to_csv("all_data.csv")
all_df.head()

The DF above is from mixing all the data together. There are a lot more GPS signals than anything else, and a lot more depth values than photos. They're all time sorted, and then the preceding value is filled down until there's another one to take over.


In [ ]:
all_df["depth"] = all_df["depth"].ffill()
all_df["depth"] = all_df["depth"].fillna(0)
all_df["filename"] = all_df["filename"].ffill(limit=10)
naieve_ffill(all_df, "geometry")
# all_df["geometry"] = all_df["geometry"].ffill()
all_df["description"] = all_df["description"].ffill()
all_df.drop(["lat", "lon"], axis=1, inplace=True, errors="ignore")
all_df.head()
# %%
all_gdf = gp.GeoDataFrame(all_df)

In [ ]:
markers_df = all_gdf[
    (all_gdf.source == "photo")
    & ((all_gdf.marker_type == "numbered") | (all_gdf.marker_number != ""))
].copy()
intermediate_df = all_gdf[
    (all_gdf.source == "photo") & (all_gdf.marker_type == "intermediate")
].copy()
markers_df.loc[:, "marker_text"] = markers_df.apply(make_marker_text, axis=1)
# %%
uni_marker_df = (
    markers_df.groupby("marker_number")
    .apply(
        lambda grp: pd.Series(
            {
                "geometry": centroid(MultiPoint(list(grp.geometry))),
                "marker_text": grp.marker_text.iloc[0],
                "depth": grp.depth.mean(),
            }
        )
    )
    .sort_index()
)
uni_marker_df

In [ ]:
fig, ax = plt.subplots(figsize=(30, 18))

# Add scalebar
starting_point = geopy_pt(-33.9175, 151.265)
scalebar_distances = [0, 5, 10, 15, 20, 50, 100]
scalebar_lines = draw_scale_bar(starting_point, scalebar_distances)
scale_gdf = gp.GeoDataFrame(geometry=scalebar_lines["scalebar_lines"])
scale_gdf.plot(ax=ax, color=TEXT_COLOUR)
for i, p in enumerate(scalebar_lines["number_points"]):
    ax.text(p.x, p.y, scalebar_distances[i], ha="center", fontsize=5, color=TEXT_COLOUR)

# Add north arrow
n_bottom_pt = geopy_pt(-33.9175, 151.267)
draw_north_arrow(ax, n_bottom_pt)

# plot the swum paths and add a colourbar
cax = all_gdf.plot(
    column="depth",
    cmap="rainbow",
    ax=ax,
    zorder=2,
    gid="depth_gps_trace",
    markersize=0.1,
)
divider = make_axes_locatable(ax)
cax_cb = divider.append_axes("right", size="2%", pad=0.05)
cbar = plt.colorbar(cax.collections[0], cax=cax_cb)
cbar.set_label("Depth")

# Plot the voronoi of bottom conditions
bottom_gdf = all_gdf[
    (all_gdf.source == "photo") & (all_gdf.bottom_condition != "unspecified")
]

# Initialize an empty list to store the filtered data
filtered_data = []

# Iterate over the GeoDataFrame
for i in range(len(bottom_gdf) - 1):
    # Swap the coordinates in the Point objects
    point1 = (bottom_gdf.iloc[i].geometry.y, bottom_gdf.iloc[i].geometry.x)
    point2 = (bottom_gdf.iloc[i + 1].geometry.y, bottom_gdf.iloc[i + 1].geometry.x)

    # Calculate the distance between the current point and the next point
    distance = geodesic(point1, point2).meters

    # If the distance is greater than or equal to 0.5 meters (500mm), keep the current point
    if distance >= 1.5:
        filtered_data.append(bottom_gdf.iloc[i])

# Append the last point as it has no next point to compare with
filtered_data.append(bottom_gdf.iloc[-1])
# Convert the list to a GeoDataFrame
filtered_gdf = gp.GeoDataFrame(pd.concat(filtered_data, axis=1).transpose())
filtered_gdf.set_crs(CRS, inplace=True)

# Load in the click data
json_df: pd.DataFrame = pd.read_json("click_conditions.json")
click_gdf = gp.GeoDataFrame(
    geometry=json_df.apply(lambda row: Point(row.lon, row.lat), axis=1)
)
click_gdf["bottom_condition"] = json_df.condition

# concat it to the bottom of the dived data
filtered_gdf = pd.concat([filtered_gdf, click_gdf])


# Convert the points in the GeoDataFrame to a MultiPoint object
points = MultiPoint(filtered_gdf.geometry.values)


# Generate the Voronoi diagram
voronoi_polygons = voronoi_diagram(points)

# Convert the Voronoi polygons to a GeoDataFrame
voronoi_gdf = gp.GeoDataFrame(
    filtered_gdf["bottom_condition"],
    geometry=list(voronoi_polygons.geoms),
    crs=bottom_gdf.crs,
)

colors = {
    "rocky": "gray",
    "sandy": "khaki",
    "kelp": "seagreen",
    "low and sandy": "darkkhaki",
    "sandy and kelp": "mediumseagreen",
    "sandy and rocky": "thistle",
    "low and rocky": "mediumaquamarine",
    "low": "lightgreen",
    "shore_rocks": "dimgrey",
    "beach": "papayawhip",
}
# temp until voronoi is sorted out
filtered_gdf["colour"] = filtered_gdf["bottom_condition"].map(colors)
filtered_gdf.plot(color=filtered_gdf.colour, ax=ax, gid="bottom_condition_markers")
# temp until voronoi is sorted out

voronoi_gdf["colour"] = voronoi_gdf["bottom_condition"].map(colors)

# Add bottom contition dots to the legend
legend_handles = []
for condition, color in colors.items():
    patch = mpatches.Patch(color=color, label=condition)
    legend_handles.append(patch)

plot_voronoi = False
if plot_voronoi:
    # TODO: The indexing between the cells and the colours is off.
    result_rows = []

    for point in filtered_gdf["geometry"]:
        # Find the polygon that contains the point
        matching_row = voronoi_gdf[voronoi_gdf.contains(point)].iloc[0]
        matching_polygon = matching_row.geometry
        result_rows.append(
            {
                "point": point,
                "polygon": matching_polygon,
                "bottom_condition": matching_row.bottom_condition,
                "colour": matching_row.colour,
            }
        )

    # Create a new GeoDataFrame
    v_and_p_gdf = gp.GeoDataFrame(result_rows, geometry="polygon")

    v_and_p_gdf.plot(
        ax=ax,
        color=v_and_p_gdf.colour,
        edgecolor=None,
        alpha=0.5,
        zorder=1,
    )

buffer_radius = 0.0003  # Set this to your desired radius
bounds = (
    Polygon(MultiPoint(filtered_gdf.geometry.values).envelope)
    .buffer(buffer_radius)
    .bounds
)
ax.set_xlim([bounds[0], bounds[2]])
ax.set_ylim([bounds[1], bounds[3]])

## end voronoi

# Contour map
contour_gdf = all_gdf.copy(deep=True)
shore_gdf = click_gdf[
    (click_gdf.bottom_condition == "shore_rocks")
    | (click_gdf.bottom_condition == "beach")
].copy()
shore_gdf["depth"] = 0
contour_gdf = pd.concat([contour_gdf, shore_gdf])
x, y, z = prep_for_contour(contour_gdf, gridpoints_x=170, gridpoints_y=70)
SHOW_CONTOUR_GRID = True
if SHOW_CONTOUR_GRID:
    ax.scatter(x, y, s=1)

norm = matplotlib.colors.Normalize(vmin=-15, vmax=0)
cmap = plt.get_cmap("rainbow")
CS = ax.contour(
    x,
    y,
    z,
    levels=list(range(-15, 1, 2)),
    colors=[cmap(norm(x)) for x in range(-15, 1, 2)],
    alpha=0.5,
)
ax.clabel(CS, inline=1, fontsize=7)


# Add context image
cx.add_basemap(ax, crs=voronoi_gdf.crs, source=cx.providers.Esri.WorldImagery)


# add the markers
intermediate_df.plot(ax=ax, marker="2", zorder=3, gid="intermediate_markers")
uni_marker_df.plot(ax=ax, marker="$\circ$", zorder=3, gid="numbered_markers")
uni_marker_df.apply(partial(add_numbered_marker_label, ax=ax), axis=1)
uni_marker_df.apply(partial(add_tolerance_circle, ax=ax), axis=1)
# intermediate_df.apply(add_intermediate_label, axis=1)
all_gdf[all_gdf.note.notnull()].apply(partial(add_note_label, ax=ax), axis=1)

# Plot the chain line.
chain_line_string = LineString(uni_marker_df.geometry)
gp.GeoDataFrame(geometry=[chain_line_string]).plot(ax=ax, linewidth=5, color="white")
total_length = measure_line_string(chain_line_string)
# print(f"The total length of the chain is {total_length:.0f} meters.")  
ax.text(
    move_pt(starting_point, 15, 0).longitude,
    starting_point.latitude,
    f"The total length of the chain is {total_length:.0f} meters. (Working number, it's more that that IRL)",
    ha="left",
    fontsize=12,
    color=TEXT_COLOUR,
)


# Add the shortcut arrows
# This should be 4 to 14, but I haven't found marker 4 yet
draw_shortcut_arrow(
    all_gdf,
    ax,
    from_marker_number=5,
    to_marker_number=14,
    text_colour=TEXT_COLOUR,
    arrow_colour=TEXT_COLOUR,
)
draw_shortcut_arrow(
    all_gdf,
    ax,
    from_marker_number=11,
    to_marker_number=16,
    text_colour=TEXT_COLOUR,
    arrow_colour=TEXT_COLOUR,
)
draw_shortcut_arrow(
    all_gdf,
    ax,
    from_marker_number=23,
    to_marker_number=5,
    text_colour=TEXT_COLOUR,
    arrow_colour=TEXT_COLOUR,
)


markers = [
    {"description": "numbered", "marker": "$\circ$", "colour": "orange"},
    {"description": "unnumbered", "marker": "2", "colour": "blue"},
]

# Create a Line2D object for each marker type and add it to the legend handles
for marker in markers:
    line = mlines.Line2D(
        [],
        [],
        color=marker["colour"],
        marker=marker["marker"],
        linestyle="None",
        markersize=10,
        label=marker["description"],
    )
    legend_handles.append(line)

tol_circle = plt.Circle(
    [], [], color="r", fill=False, alpha=0.4, linestyle="-.", label="Tolerance area"
)
legend_handles.append(tol_circle)
ax.legend(handles=legend_handles, loc="lower left")
ax.set_title("Gordon's bay trail, coloured by depth")
plt.tight_layout()
plt.savefig("docs/marker_graph.png")
# plt.savefig("docs/marker_graph.svg")
# print(all_gdf[all_gdf.source == "photo"].shape[0], "photos")

In [ ]:
# %% folium map time
gordons_coords = [-33.9161117, 151.26369831]

f_map = folium.Map(
    location=gordons_coords,
    # tiles="CartoDB Positron",
    tiles="https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}",
    attr="Esri",
    name="Esri Satellite",
    zoom_start=17,
)
dive_1 = folium.FeatureGroup(name="chain trail, depth and gps", show=False).add_to(
    f_map
)
dive_2 = folium.FeatureGroup(name="boulder garden, depth and gps", show=False).add_to(
    f_map
)
dive_3 = folium.FeatureGroup(
    name="along the bottom of the wall's rocks, depth and gps", show=False
).add_to(f_map)
dive_4 = folium.FeatureGroup(
    name="across to the other side and back, depth and gps", show=False
).add_to(f_map)

markers = folium.FeatureGroup(name="markers", show=True).add_to(f_map)
intermediate_markers = folium.FeatureGroup(
    name="intermediate markers", show=True
).add_to(f_map)
trail = folium.FeatureGroup(name="the trail", show=True).add_to(f_map)


def add_depth_trace_to_map(
    all_gdf, to_add_to, filter_name="chain_loop", depth_filter=-0.3, radius=2
):
    for index, row in all_gdf[
        (all_gdf.description == filter_name) & (all_gdf.depth < depth_filter)
    ].iterrows():
        time = row.name.astimezone(pytz.timezone("Australia/Sydney")).strftime(
            "%H:%M:%S"
        )
        tt = f"{time} {row.depth}"
        folium.CircleMarker(
            location=[row.geometry.y, row.geometry.x],
            radius=radius,
            color=depth_to_colour(row.depth, all_gdf),
            stroke=False,
            fill=True,
            weight=3,
            fill_opacity=0.6,
            opacity=1,
            tooltip=tt,
        ).add_to(to_add_to)


add_depth_trace_to_map(all_gdf, dive_1, filter_name="chain_loop")
add_depth_trace_to_map(all_gdf, dive_2, filter_name="boulder_garden", radius=1)
add_depth_trace_to_map(all_gdf, dive_3, filter_name="wall_to_desert", radius=1)
add_depth_trace_to_map(all_gdf, dive_4, filter_name="far_side_desert", radius=1)

folium.PolyLine(
    locations=[[p.y, p.x] for p in uni_marker_df.geometry],
    color="white",
    weight=2,
    tooltip="The trail, as it stands now",
).add_to(trail)

for index, row in uni_marker_df.iterrows():
    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius=5,
        color="white",
        stroke=True,
        fill=True,
        weight=1,
        fill_opacity=0.1,
        opacity=1,
        tooltip=row.marker_text,
    ).add_to(markers)

for index, row in intermediate_df.iterrows():
    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius=2,
        color="white",
        stroke=True,
        fill=True,
        weight=1,
        fill_opacity=0.1,
        opacity=1,
        tooltip=row.filename,
    ).add_to(intermediate_markers)

folium.LayerControl().add_to(f_map)
f_map.save("docs/gordons_map.html")

f_map

In [ ]:
%matplotlib notebook
from ipywidgets import interact
import matplotlib.pyplot as plt
import numpy as np

vol = np.random.uniform(size=(16, 16, 16))

fig, ax = plt.subplots()
im = ax.imshow(vol[0])

@interact(z=(0, 15))
def show(z):
    im.set_array(vol[z])
    im.set_clim(vol[z].min(), vol[z].max())
    fig.canvas.draw_idle()